# The dataset

The IMDB sentiment dataset is a collection of 50K movie reviews, annotated as positive or negative, and split in two sets of equal size: a training and a test set. Both set have an equal number of positive and negative review. The dataset is available on several libraries, but we ask that you use the HuggingFace [datasets](https://huggingface.co/datasets/imdb) version. Follow their [tutorial](https://huggingface.co/docs/datasets/load_hub) on how to use the library for more details.

Download and look at the dataset, and answer the following questions.
1. How many splits does the dataset has? (1 point)
2. How big are these splits? (1 point)
3. What is the proportion of each class on the supervised splits? (1 point)

## Installation and import of necessary librairies


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 432 kB 2.1 MB/s 
     |████████████████████████████████| 115 kB 49.7 MB/s 
     |████████████████████████████████| 212 kB 30.7 MB/s 
     |████████████████████████████████| 163 kB 35.8 MB/s 
     |████████████████████████████████| 127 kB 34.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 1.7 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3166303 sha256=1ab015abe678bbffc4b1f21a1be55cd5c8b09cb01cce48b79728dc06b1b33513
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
!python -m spacy download en_core_web_sm

2022-10-11 14:13:46.300952: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 2.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
from datasets import load_dataset_builder
from datasets import load_dataset
import pandas as pd
import numpy as np

import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize 
import re
import spacy

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline

database_name = "imdb"
ds_builder = load_dataset_builder(database_name)
print(ds_builder.info.description)
print(ds_builder.info.features)

dataset = load_dataset(database_name)

from datasets import get_dataset_split_names
import fasttext

Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.
{'text': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['neg', 'pos'], id=None)}


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Import of our file with usefull functions
%load utility_functions.py
import sys
sys.path.append("../tools/")
import utility_functions

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## The dataset

Now that we installed all the necessary librairies, let's have a look on our data.

In [ ]:
print("Split names", get_dataset_split_names(database_name))
dataset

Split names ['train', 'test', 'unsupervised']


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

We can see that this database has 3 splits. The "train" and "test" splits have 25000 rows each and the unsupervised split has 50000 rows.

As here we are only going to do supervised learning, we will only have two datasets : train and test.

In [ ]:
# To start we are going to split our datasets into 3 differents datasets
train = dataset["train"].to_pandas()
test = dataset["test"].to_pandas()

# Then we will have a look on the 
print("Test values count : {0}".format(len(test)))
print(test["label"].value_counts())
print("Train values count : {0}".format(len(train)))
print(train["label"].value_counts())

Test values count : 25000
0    12500
1    12500
Name: label, dtype: int64
Train values count : 25000
0    12500
1    12500
Name: label, dtype: int64


As we can also see it here, we have an equal amount of each different label (0 and 1) in our datasets.

Let's now have a look on our dataframe and our data 

In [ ]:
train.head()

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


Now we will do a quick shuffle of our dataset to make sure to not have any order in our data.

## Preprocessing

But before doing anything else with our dataset, we must preprocess it by changing all letters to lowercase and removing punctuation.

In [ ]:
train = utility_functions.preprocess_df(train)
test = utility_functions.preprocess_df(test)

train.head()

,text,label
0,i rented i am curiousyellow from my video stor...,0
1,i am curious yellow is a risible and pretentio...,0
2,if only to avoid making this type of film in t...,0
3,this film was probably inspired by godards mas...,0
4,oh brotherafter hearing about this ridiculous ...,0


As we can see it, our preprocess is now done !

## Shuffling the dataset

To get more coherent results, is is necessary to shuffle the dataset (we will also need it for the hyperparameter optimization later) as follow:

In [ ]:
# Function used to shuffle the given dataset
def shuffle(dataset):
  return dataset.sample(frac=1).reset_index(drop=True)

In [ ]:
train = shuffle(train)
test = shuffle(test)

# Displaying a dataset to see if the shuffle worked
train.head()

,text,label
0,i watched this movie because i like nicolas ca...,0
1,manoj agrawal after the failure of pardesi bab...,0
2,if this film was a comedy i would have given i...,0
3,for starters i didnt even know about this show...,1
4,this is the epitome of fairytale the villains ...,1


We can see that the rows of the datasets have indeed been shuffled, as compared to their previous order.

## 1 - (2 points) Turn the dataset into a dataset compatible with Fastext (see the Tips on using FastText section a bit lower)

The FastText dataset format is:

```__label__<your_label> <corresponding text>```

For convenience, we will replace the original labels (0 and 1) by their intended meaning, "negative" and "positive" respectively.

In [ ]:
def pandas_to_fasttext(pandas_data):
  '''Convert a pandas dataset to a string of fasttext format'''
  pandas_data = pandas_data.astype('str')
  fasttext_data = '__label__' + pandas_data['label'] + ' ' + pandas_data['text']
  fasttext_str = fasttext_data.to_string(index=False)
  fasttext_str = fasttext_str.replace('0', 'negative').replace('1', 'positive')
  return fasttext_str

def str_to_txt_file(text, filename):
  '''Saves a string as a txt file'''
  file = open(filename, "w")
  file.write(text)
  file.close()

We now convert each dataset (train, test) to fasttext format using the above functions, and save them to text files for later use:

In [ ]:
fasttext_train = pandas_to_fasttext(train)
str_to_txt_file(fasttext_train, 'train.txt')

fasttext_test = pandas_to_fasttext(test)
str_to_txt_file(fasttext_test, 'test.txt')

fasttext_train.split('\n')[:10]
print(fasttext_train.split('\n')[:1])

['__label__negative i watched this movie because i like ...']


We can see above the result of fasttext formatting, which correspond to the expected format.

## 2 - (2 points) Train a FastText classifier with default parameters on the training data, and evaluate it on the test data using accuracy.

Training simple classifier with formatted train dataset:

In [ ]:
model = fasttext.train_supervised(input='train.txt')

Displaying the results of the model on the formatted test dataset:

In [ ]:
test_results = model.test("test.txt")
print('Number of samples:', test_results[0])
print('Accurracy:', test_results[1])

Number of samples: 25000
Accurracy: 0.6396


## 3 - (3 points) Use the hyperparameters search functionality of FastText and repeat step 2

Created the hyper_validation dataset by taking half of the test dataset:

In [ ]:
test_size = len(test)//2

hyper_validation = test.iloc[test_size:,:]

display(hyper_validation)

,text,label
12500,right up or down there with toys and jurassic ...,0
12501,the monkees surprisingly are a big favorite of...,1
12502,this movie is simply rubbish i have to say i a...,0
12503,stephen hawking has one of the greatest minds ...,1
12504,one of the worst excuses for an irish accents ...,0
...,...,...
24995,this is one of the very very few films that ar...,1
24996,this movie is just great it starts out real sl...,1
24997,this is horrible even for a tv movie i cant be...,0
24998,curacao is a foreign intrigue drama set on the...,0


Formatting this validation dataset to fasttext format and saving it to txt file:

In [ ]:
fasttext_validation = pandas_to_fasttext(hyper_validation)
str_to_txt_file(fasttext_validation, 'hyper_validation.txt')

fasttext_validation.split('\n')[:10]

['__label__negative right up or down there with toys and...',
 '__label__positive the monkees surprisingly are a big f...',
 '__label__negative this movie is simply rubbish i have ...',
 '__label__positive stephen hawking has one of the great...',
 '__label__negative one of the worst excuses for an iris...',
 '__label__positive when i sat down to watch greek for t...',
 '__label__negative i never comment on a film but i have...',
 '__label__positive for the record the positive949 version of t...',
 '__label__negative this weeks surprise screening at gv ...',
 '__label__negative let me just give you guys some advic...']

Training the hyperparameter optimized model with the train and validation dataset:

In [ ]:
hyper_model = fasttext.train_supervised(input='train.txt', autotuneValidationFile='hyper_validation.txt')

Displaying optimized model results:

In [ ]:
hyper_test_results = hyper_model.test("test.txt")
print('Number of samples:', hyper_test_results[0])
print('Accurracy:', hyper_test_results[1])

Number of samples: 25000
Accurracy: 0.6448


## 4 - (1 points) Look at the differences between the default model and the attributes found with hyperparameters search. How do the two models differ? 

Displaying the two models performances on the same test dataset:

In [ ]:
print('Number of samples:', hyper_test_results[0])
print('Accurracy (without hyperparameters search):\n', test_results[1])
print('Accurracy (with hyperparameters search):\n', hyper_test_results[1])

Number of samples: 25000
Accurracy (without hyperparameters search):
 0.6396
Accurracy (with hyperparameters search):
 0.6448


We can see a good improvement of the model accurracy when using hyperparameters search (0.6396 and 0.6448 respectively) on the same test dataset.

## 5 - (1 point) Using the tuned model, take at least 2 wrongly classified examples from the test set, and try explaining why the model failed.

Getting all predictions of the tuned model and saving it to 'prediction' variable:

In [ ]:
predictions = hyper_model.predict(test['text'].tolist())

Displaying the actual versus predicted labels with associated inputs:

In [ ]:
print('\nActual values:')
display(fasttext_test.split('\n')[:5])

print('\nPredictions:')
display(predictions[0][:5])

print('\nThe model guessed wongly on the following inputs:')

print('- Error 1: Was positive, guessed negative:')
print(test.text[1].replace('<br /><br />', ' '))

print('\n- Error 2: Was negative, guessed positive:')
print(test.text[5].replace('<br /><br />', ' '))


Actual values:


['__label__positive pulling in 26 million viewers one ha...',
 '__label__negative the first movie of this series was w...',
 '__label__negative did someone find the plot somewhere ...',
 '__label__positive we all know what chanwook park can d...',
 '__label__positive the violent death of fernando ramos ...']


Predictions:


[['__label__negative'],
 ['__label__negative'],
 ['__label__negative'],
 ['__label__positive'],
 ['__label__positive']]


The model guessed wongly on the following inputs:
- Error 1: Was positive, guessed negative:
the first movie of this series was well written and original this show drags on poorly written gags boring flashbacks not the comedy that i expected even the young folks found it boring there are certainly bright moments historical elements and some good acting but overall i can only recommend this for dvdtape at home

- Error 2: Was negative, guessed positive:
if you want to see what could be classed as the stage version of gypsy this is the film for youbr br if you enjoyed listing to the merman recording of gypsy then you really enjoy watching midler as gypsys mother rose its my opinion that midler has the volume vibrato and presents that merman once hadbr br its not often these days when listening to update versions of musicals that i get that tingling sensation that makes the hair on my neck tingle but bette midler certainly shows her talents in this movie see how you like them applesbr br

The model guessed wongly on the following inputs:
- Error 1: Was positive, guessed negative:
the first movie of this series was well written and original this show drags on poorly written gags boring flashbacks not the comedy that i expected even the young folks found it boring there are certainly bright moments historical elements and some good acting but overall i can only recommend this for dvdtape at home

- Error 2: Was negative, guessed positive:
if you want to see what could be classed as the stage version of gypsy this is the film for youbr br if you enjoyed listing to the merman recording of gypsy then you really enjoy watching midler as gypsys mother rose its my opinion that midler has the volume vibrato and presents that merman once hadbr br its not often these days when listening to update versions of musicals that i get that tingling sensation that makes the hair on my neck tingle but bette midler certainly shows her talents in this movie see how you like them applesbr br i know you may not like it but for me ms midler is the definitive gypsy

We can see that the model guessed wrongly on these inputs.

After analysis, it seems that theses inputs contains words that the model probably uses to determine whether or not an input is positive:

In input 1, the input is 'positive' but the model guessed 'negative'.
This is probably because the following negative words are present in it:
- poorly written
- boring flashbacks
- found it boring

We can also notice that even from a human point of view the review would probably seem negative, which is probably a good part of why the model guessed wrongly.

In input 2, the input is 'negative' but the model guessed 'positive'.
This is probably because the following positive words are present in it:
- you really enjoy
- shows her talent
- you like them

For probably the same reason as the previous one, the model guessed wrong there too.